In [1]:
# code code code 

from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path
from functools import wraps

In [2]:
BASE_DIR = Path("../ma-data/ma/enrollment/Extracted Data")


def monthlist_for_year(y: int) -> list[str]:
    months = range(1, 13) if y == 2018 else range(1, 13)
    return [f"{m:02d}" for m in months]

In [3]:
CONTRACT_COLUMNS = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
]
CONTRACT_DTYPES = {
    "contractid": "string",
    "planid": "float64",
    "org_type": "string",
    "plan_type": "string",
    "partd": "string",
    "snp": "string",
    "eghp": "string",
    "org_name": "string",
    "org_marketing_name": "string",
    "plan_name": "string",
    "parent_org": "string",
    "contract_date": "string",
}

In [4]:
ENROLL_COLUMNS = [
    "contractid", "planid", "ssa", "fips", "state", "county", "enrollment"
]
ENROLL_DTYPES = {
    "contractid": "string",
    "planid": "float64",
    "ssa": "float64",
    "fips": "float64",
    "state": "string",
    "county": "string",
    "enrollment": "float64",
}


def read_contract(path: Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=CONTRACT_COLUMNS,
        dtype=CONTRACT_DTYPES,
        low_memory=False,
        encoding="utf-8",
        encoding_errors="replace",
    )



def read_enroll(path: Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=ENROLL_COLUMNS,
        dtype=ENROLL_DTYPES,
        na_values=["*"],
        low_memory=False,
        encoding="utf-8",
        encoding_errors="replace",
    )


In [5]:
def fill_downup(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    df[cols] = df[cols].ffill().bfill()
    return df



def load_month(m: str, y: int) -> pd.DataFrame:
    c_path = BASE_DIR / f"CPSC_Contract_Info_{y}_{m}.csv"
    e_path = BASE_DIR / f"CPSC_Enrollment_Info_{y}_{m}.csv"

    contract_info = (
        read_contract(c_path)
        .drop_duplicates(subset=["contractid", "planid"], keep="first")
    )

    enroll_info = read_enroll(e_path)

    return (
        contract_info
        .merge(enroll_info, on=["contractid", "planid"], how="left")
        .assign(month=int(m), year=y)
    )


def build_plan_year(y: int) -> pd.DataFrame:
    monthlist = monthlist_for_year(y)

    plan_year = pd.concat(
        [load_month(m, y) for m in monthlist],
        ignore_index=True
    )

    plan_year = plan_year.sort_values(
        ["contractid", "planid", "state", "county", "month"],
        kind="mergesort"
    )

    plan_year = (
        plan_year
        .groupby(["state", "county"], dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["fips"]))
        .groupby(["contractid", "planid"], dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["plan_type", "partd", "snp", "eghp", "plan_name"]))
        .groupby("contractid", dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["org_type", "org_name", "org_marketing_name", "parent_org"]))
        .reset_index(drop=True)
    )
    return plan_year

In [6]:
def collapse_to_yearly_panel(plan_year: pd.DataFrame) -> pd.DataFrame:
    plan_year = plan_year.sort_values(
        ["contractid", "planid", "fips", "year", "month"],
        kind="mergesort"
    )

    def summarize(g):
        enroll = g["enrollment"]
        nonmiss = enroll.notna().sum()
        vals = enroll.dropna()
        
        return pd.Series({
            "n_nonmiss": nonmiss,
            "avg_enrollment": vals.mean() if nonmiss > 0 else np.nan,
            "sd_enrollment": vals.std(ddof=1) if nonmiss > 1 else np.nan,
            "min_enrollment": vals.min() if nonmiss > 0 else np.nan,
            "max_enrollment": vals.max() if nonmiss > 0 else np.nan,
            "first_enrollment": vals.iloc[0] if nonmiss > 0 else np.nan,
            "last_enrollment": vals.iloc[-1] if nonmiss > 0 else np.nan,
            "state": g["state"].iloc[-1],
            "county": g["county"].iloc[-1],
            "org_type": g["org_type"].iloc[-1],
            "plan_type": g["plan_type"].iloc[-1],
            "partd": g["partd"].iloc[-1],
            "snp": g["snp"].iloc[-1],
            "eghp": g["eghp"].iloc[-1],
            "org_name": g["org_name"].iloc[-1],
            "org_marketing_name": g["org_marketing_name"].iloc[-1],
            "plan_name": g["plan_name"].iloc[-1],
            "parent_org": g["parent_org"].iloc[-1],
            "contract_date": g["contract_date"].iloc[-1],
        })

    return (
        plan_year
        .groupby(["contractid", "planid", "fips", "year"], dropna=False)
        .apply(summarize)
        .reset_index()
    )

if __name__ == "__main__":
    y = 2018
    plan_year = build_plan_year(y)
    final_plans = collapse_to_yearly_panel(plan_year)

/tmp/ipykernel_1251275/495587463.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_downup(g, ["fips"]))
/tmp/ipykernel_1251275/495587463.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_downup(g, ["plan_type", "partd", "snp", "eghp", "plan_name"]))
/tmp/ipykernel_1251275/495587463.py:46: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This beh

In [7]:

## Service area


def monthlist_for_year(y: int) -> list[str]:
    months = range(1, 13) if y == 2018 else range(1, 13)
    return [f"{m:02d}" for m in months]


SA_COLUMNS = [
    "contractid","org_name","org_type","plan_type","partial","eghp",
    "ssa","fips","county","state","notes"
]

SA_DTYPES = {
    "contractid": "string",
    "org_name":   "string",
    "org_type":   "string",
    "plan_type":  "string",
    "partial":    "boolean",
    "eghp":       "string",
    "ssa":        "float64",
    "fips":       "float64",
    "county":     "string",
    "state":      "string",
    "notes":      "string",
}

def read_service_area(path: str | Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=SA_COLUMNS,
        na_values=["*"],
        dtype=SA_DTYPES,
        low_memory=False,
    )


def load_month_sa(m: str, y: int) -> pd.DataFrame:
    path = Path("../ma-data/ma/service-area/Extracted Data") / f"MA_Cnty_SA_{y}_{m}.csv"
    df = read_service_area(path)
    df["month"] = int(m)
    df["year"] = y
    return df


def build_final_service_area(y: int) -> tuple[pd.DataFrame, pd.DataFrame]:
    
    monthlist = monthlist_for_year(y)
    service_year = pd.concat((load_month_sa(m, y) for m in monthlist), ignore_index=True)

    service_year = service_year.sort_values(
        ["contractid", "fips", "state", "county", "month"],
        kind="mergesort"  # stable
    )

    service_year["fips"] = (
        service_year.groupby(["state", "county"], dropna=False)["fips"]
        .transform(lambda s: s.ffill().bfill())
    )

    # R: group_by(contractid) %>% fill(plan_type, partial, eghp, org_type, org_name, .direction="downup")
    fill_cols = ["plan_type", "partial", "eghp", "org_type", "org_name"]
    service_year[fill_cols] = (
        service_year.groupby(["contractid"], dropna=False)[fill_cols]
        .transform(lambda df: df.ffill().bfill())
    )

    service_year = service_year.sort_values(["contractid", "fips", "year", "month"], kind="mergesort")
    final_service_area = (
        service_year.groupby(["contractid", "fips", "year"], dropna=False, as_index=False)
        .agg(
            state=("state", "last"),
            county=("county", "last"),
            org_name=("org_name", "last"),
            org_type=("org_type", "last"),
            plan_type=("plan_type", "last"),
            partial=("partial", "last"),
            eghp=("eghp", "last"),
            ssa=("ssa", "last"),
            notes=("notes", "last"),
        )
    )

    return service_year, final_service_area



In [8]:
service_year, final_service_area = build_final_service_area(2018)
print(service_year.shape, final_service_area.shape)
print(final_service_area.head())


(3986459, 13) (331593, 12)
  contractid     fips  year state    county  \
0      90091      NaN  2018  <NA>      <NA>   
1      H0022  39023.0  2018    OH     Clark   
2      H0022  39035.0  2018    OH  Cuyahoga   
3      H0022  39051.0  2018    OH    Fulton   
4      H0022  39055.0  2018    OH    Geauga   

                                           org_name          org_type  \
0  UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT  HCPP - 1833 Cost   
1               BUCKEYE COMMUNITY HEALTH PLAN, INC.              Demo   
2               BUCKEYE COMMUNITY HEALTH PLAN, INC.              Demo   
3               BUCKEYE COMMUNITY HEALTH PLAN, INC.              Demo   
4               BUCKEYE COMMUNITY HEALTH PLAN, INC.              Demo   

                           plan_type  partial  eghp      ssa  \
0                   HCPP - 1833 Cost     <NA>  <NA>      NaN   
1  Medicare-Medicaid Plan HMO/HMOPOS     <NA>  <NA>  36110.0   
2  Medicare-Medicaid Plan HMO/HMOPOS     <NA>  <NA>  36170.

In [9]:
final_service_area.head()

,contractid,fips,year,state,county,org_name,org_type,plan_type,partial,eghp,ssa,notes
0,90091,NaN,2018,<NA>,<NA>,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,HCPP - 1833 Cost,HCPP - 1833 Cost,<NA>,<NA>,NaN,"Covers the entire US, all States and Counties"
1,H0022,39023.0,2018,OH,Clark,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,<NA>,<NA>,36110.0,<NA>
2,H0022,39035.0,2018,OH,Cuyahoga,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,<NA>,<NA>,36170.0,<NA>
3,H0022,39051.0,2018,OH,Fulton,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,<NA>,<NA>,36260.0,<NA>
4,H0022,39055.0,2018,OH,Geauga,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,<NA>,<NA>,36280.0,<NA>


In [10]:
final_plans.head()

,contractid,planid,fips,year,n_nonmiss,avg_enrollment,sd_enrollment,min_enrollment,max_enrollment,first_enrollment,...,org_type,plan_type,partd,snp,eghp,org_name,org_marketing_name,plan_name,parent_org,contract_date
0,90091,NaN,NaN,2018,0,NaN,NaN,NaN,NaN,NaN,...,HCPP - 1833 Cost,HCPP - 1833 Cost,No,No,No,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,United Mine Workers of America Health & Retire...,<NA>,UMWA Health and Retirement Funds,02/01/1974 0:00:00
1,E0654,801.0,1001.0,2018,0,NaN,NaN,NaN,NaN,NaN,...,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust (Employe...,IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00
2,E0654,801.0,1003.0,2018,12,13.833333,1.029857,13.0,15.0,13.0,...,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust (Employe...,IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00
3,E0654,801.0,1005.0,2018,0,NaN,NaN,NaN,NaN,NaN,...,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust (Employe...,IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00
4,E0654,801.0,1007.0,2018,0,NaN,NaN,NaN,NaN,NaN,...,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust (Employe...,IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00


In [11]:
print(final_plans.head())

  contractid  planid    fips  year  n_nonmiss  avg_enrollment  sd_enrollment  \
0      90091     NaN     NaN  2018          0             NaN            NaN   
1      E0654   801.0  1001.0  2018          0             NaN            NaN   
2      E0654   801.0  1003.0  2018         12       13.833333       1.029857   
3      E0654   801.0  1005.0  2018          0             NaN            NaN   
4      E0654   801.0  1007.0  2018          0             NaN            NaN   

   min_enrollment  max_enrollment  first_enrollment  ...  \
0             NaN             NaN               NaN  ...   
1             NaN             NaN               NaN  ...   
2            13.0            15.0              13.0  ...   
3             NaN             NaN               NaN  ...   
4             NaN             NaN               NaN  ...   

                                  org_type  \
0                         HCPP - 1833 Cost   
1  Employer/Union Only Direct Contract PDP   
2  Employer/Union On

In [12]:
print("FINAL_PLANS OVERVIEW")
print("-" * 40)
print("Shape (rows, columns):", final_plans.shape)
print("\nColumn dtypes:")
print(final_plans.dtypes)

numeric_cols = [
    "avg_enrollment",
    "sd_enrollment",
    "min_enrollment",
    "max_enrollment",
    "first_enrollment",
    "last_enrollment",
]

print("\nNumeric summary statistics:")
print(final_plans[numeric_cols].describe())

print("\nCoverage checks:")
print("Unique contracts:", final_plans["contractid"].nunique())
print("Unique plans:", final_plans[["contractid", "planid"]].drop_duplicates().shape[0])
print("Unique counties (FIPS):", final_plans["fips"].nunique())
print("Years covered:", sorted(final_plans["year"].unique()))


FINAL_PLANS OVERVIEW
----------------------------------------
Shape (rows, columns): (2475118, 23)

Column dtypes:
contractid            string[python]
planid                       float64
fips                         float64
year                           int64
n_nonmiss                      int64
avg_enrollment               float64
sd_enrollment                float64
min_enrollment               float64
max_enrollment               float64
first_enrollment             float64
last_enrollment              float64
state                         object
county                        object
org_type                      object
plan_type                     object
partd                         object
snp                           object
eghp                          object
org_name                      object
org_marketing_name            object
plan_name                     object
parent_org                    object
contract_date                 object
dtype: object

Numeric summary sta

In [19]:
print("FINAL_SERVICE_AREA OVERVIEW")
print("-" * 40)
print("Shape (rows, columns):", final_service_area.shape)
print("\nColumn dtypes:")
print(final_service_area.dtypes)

sa_numeric_cols = ["ssa", "fips"]
print("\nNumeric summary statistics:")
print(final_service_area[sa_numeric_cols].describe())

print("\nCoverage checks:")
print("Unique contracts:", final_service_area["contractid"].nunique())
print("Unique counties (FIPS):", final_service_area["fips"].nunique())
print("Unique contract-county-year rows:", final_service_area[["contractid", "fips", "year"]].drop_duplicates().shape[0])
print("Years covered:", sorted(final_service_area["year"].unique()))



print("\nPlan types by row count:")
print(final_service_area["plan_type"].value_counts().head(15))




FINAL_SERVICE_AREA OVERVIEW
----------------------------------------
Shape (rows, columns): (331593, 12)

Column dtypes:
contractid    string[python]
fips                 float64
year                   int64
state         string[python]
county        string[python]
org_name      string[python]
org_type      string[python]
plan_type     string[python]
partial              boolean
eghp          string[python]
ssa                  float64
notes         string[python]
dtype: object

Numeric summary statistics:
                 ssa           fips
count  331547.000000  331547.000000
mean    28473.856811   31223.788962
std     14732.795671   15768.079707
min      1000.000000    1001.000000
25%     16170.000000   19035.000000
50%     27270.000000   30055.000000
75%     42280.000000   46053.000000
max     65220.000000   78030.000000

Coverage checks:
Unique contracts: 716
Unique counties (FIPS): 3313
Unique contract-county-year rows: 331593
Years covered: [np.int64(2018)]

Plan types by row cou

In [14]:
# Question 1

table1 = (
    final_plans
    .drop_duplicates(subset=["contractid", "planid"])  
    .groupby("plan_type")
    .size()
    .reset_index(name="Number of Plans")
    .sort_values("Number of Plans", ascending=False)
)

table1


,plan_type,Number of Plans
3,HMO/HMOPOS,2678
6,Medicare Prescription Drug Plan,1011
4,Local PPO,966
8,National PACE,258
10,Regional PPO,109
0,1876 Cost,101
7,Medicare-Medicaid Plan HMO/HMOPOS,54
9,PFFS,50
2,HCPP - 1833 Cost,9
5,MSA,5


In [15]:
# Question 2

# remove SNP, egph, and 800-series (800-899 planid )
filtered_plans = (
    final_plans
    # remove SNP plans
    .query("snp != 'Yes'")
    # remove EGHP plans
    .query("eghp != 'Yes'")
    # remove 800-series plans
    .query("~(planid >= 800 and planid < 900)")
)


In [16]:
table1_updated = (
    filtered_plans
    .drop_duplicates(subset=["contractid", "planid"])
    .groupby("plan_type")
    .size()
    .reset_index(name="Number of Plans")
    .sort_values("Number of Plans", ascending=False)
)

table1_updated


,plan_type,Number of Plans
2,HMO/HMOPOS,1569
5,Medicare Prescription Drug Plan,794
3,Local PPO,569
7,National PACE,258
0,1876 Cost,93
6,Medicare-Medicaid Plan HMO/HMOPOS,54
9,Regional PPO,49
8,PFFS,46
1,HCPP - 1833 Cost,9
4,MSA,3


In [17]:
# Question 3

# Merge dataset - use inner to remove to retain filtering from Q2
plans_with_service_area = (
    filtered_plans
    .merge(
        final_service_area[["contractid", "fips", "year"]],
        on=["contractid", "fips", "year"],
        how="inner"   # THIS enforces “approved counties only”
    )
)



In [18]:
table3 = (
    plans_with_service_area
    .drop_duplicates(subset=["contractid", "planid", "fips", "year"])
    .groupby("plan_type")["avg_enrollment"]
    .mean()
    .reset_index(name="Average Enrollment")
    .sort_values("Average Enrollment", ascending=False)
)

table3


,plan_type,Average Enrollment
5,Medicare-Medicaid Plan HMO/HMOPOS,989.168755
2,HMO/HMOPOS,755.549628
3,Local PPO,330.622892
0,1876 Cost,251.565223
8,Regional PPO,188.788397
6,National PACE,144.327947
7,PFFS,93.659225
4,MSA,58.131918
1,HCPP - 1833 Cost,NaN
